In [ ]:
import glob
import numpy as np
import pandas as pd
import sys
from matplotlib import pyplot as plt
sys.path.insert(1, './')
import functions

In [ ]:
#reading in file


filename = sorted(glob.glob("p0.0_n20_rcut2.5_T0.50_run1/run_0[1-2]*.dump"))

data, nlin, nlog, l_box, time = functions.read_lammps(filename)
data = np.reshape(data, (nlin,nlog, data.shape[1], data.shape[2]))
#removing the first conf from each loop
data = data[:,1:,:,:]
nlog = nlog -1
print(data.shape, nlin, nlog)

# ordering the ids of the particles
for i in range(nlin):
    for j in range(nlog):
        data[i,j,:,:] = data[i,j,data[i,j,:,0].astype(int).argsort(),:]

id_a, type_a, mol_a, pos_a, vel_a = data[:,:,:,0].astype(int), data[:,:,:,1].astype(int), data[:,:,:,2].astype(int), data[:,:,:,3:6].astype(float), data[:,:,:,6:9].astype(float)  
# mol_a is a 1d array with the n 
del data
pos_a = pos_a.astype(float)
l_box = l_box.astype(float)

#pos_a = np.reshape(pos_a, (nlin*nlog, pos_a.shape[2], pos_a.shape[3]))
pos_a_orig = pos_a.copy()
mol_a_orig = mol_a.copy()


In [ ]:
# remove the COM drift
pos_a = pos_a_orig.copy()
mol_a = mol_a_orig.copy()

print(pos_a.shape)
pos_a = functions.unwrap_pos(pos_a,l_box[0,:])
print(pos_a.shape)



#test the drift code by adding artifitial drift
if (len(pos_a.shape)==4):
    pos_a = np.reshape(pos_a, (nlin*nlog, pos_a.shape[2], pos_a.shape[3]))
pid = 1
# plot the position before drift
plt.plot(pos_a[:,pid,1])

#create the drift
drift = np.arange(0.,2.,2./420.)
drift = np.expand_dims(drift, axis=1)
drift = np.expand_dims(drift, axis=1)
print(drift.shape)
# add drift
pos_a[:,:,1] = pos_a[:,:,1] + drift[:,:,0] 

pid = 1
# plot the position after drift
plt.plot(pos_a[:,pid,1])
      
      
pos_a = functions.removing_com_drift(pos_a,l_box[0,:])
pos_a = np.reshape(pos_a, (nlin, nlog, pos_a.shape[1], pos_a.shape[2]))

pos_a = np.reshape(pos_a, (nlin*nlog, pos_a.shape[2], pos_a.shape[3]))
mol_a = np.reshape(mol_a, (nlin*nlog, mol_a.shape[2]))

pid = 1
# plot the position after fixing the drift
plt.plot(pos_a[:,pid,1])
#plt.plot(pos_a[:,pid,0], pos_a[:,pid,1])
pos_a_orig = pos_a.copy()

In [ ]:
#calculating the gyration tensor and rg
gyration = functions.gTensor(pos_a, l_box[0,:], mol_a)
gyration = np.reshape(gyration, (gyration.shape[0]*gyration.shape[1], gyration.shape[2]))
np.save("gyration_tensor",gyration)
gyration = np.load("gyration_tensor.npy")
rg_val, eigen_val, eigen_vec = functions.rg(gyration)
np.save("rg",rg_val)
np.save("eigenval",eigen_val)
np.save("eigenvec",eigen_vec)


In [ ]:
# calculate translational MSD
pos_a = np.reshape(pos_a, (nlin, nlog, pos_a.shape[1], pos_a.shape[2]))
#pos_a = pos_a_orig.copy()
print(np.array(time).shape)
print(pos_a.shape)
log_space = 2. # the base for the log scpaced time log_space^n
msd_log, alpha2_log, msd_time_log = functions.msd_log(pos_a,l_box[0,:],log_space)
print(pos_a.shape)
msd_lin, alpha2_lin, msd_time_lin = functions.msd_lin(pos_a,l_box[0,:],log_space)




In [ ]:
from matplotlib import pyplot as plt

print(msd_log.shape, msd_lin.shape)
msd = np.concatenate((msd_log, msd_lin), axis=0)
msd_time = np.concatenate((msd_time_log, msd_time_lin), axis=0)
plt.plot(msd_time,msd, marker="o")
plt.xscale("log"); plt.yscale("log")
msd = np.transpose(np.vstack((msd_time, msd)))
df = pd.DataFrame(msd)
df.columns=["time", "msd"]
df.to_pickle("msd")
np.save("msd", msd)



In [ ]:
# calculating the fqt_self
Q = 7.1
log_space = 2
pos_a = np.reshape(pos_a_orig, (nlin, nlog, pos_a.shape[1], pos_a.shape[2]))

fqt_lin, time_lin = functions.fqt_self_lin(pos_a[:10,:,:,:],l_box[0,:],Q, log_space)
fqt_log, time_log = functions.fqt_self_log(pos_a[:10,:,:,:],l_box[0,:],Q, log_space)
#fqt_log_2, time_log_2 = functions.fqt_self_log_2(pos_a[:10,:,:,:],l_box[0,:],Q, log_space)



In [ ]:
from matplotlib import pyplot as plt

print(fqt_log.shape, fqt_lin.shape)
fqt = np.concatenate((fqt_log, fqt_lin), axis=0)
fqt_time = np.concatenate((time_log, time_lin), axis=0)
plt.scatter(fqt_time,fqt, marker="o")
plt.xscale("log"); #plt.yscale("log")
fqt = np.transpose(np.vstack((fqt_time, fqt)))
df = pd.DataFrame(fqt)
df.columns=["time", "fqt"]
df.to_pickle("fqt")
np.save("msd", fqt)



In [ ]:
def fqt_self_log(all_pos,lbox,Q, log_space):
    nlin=all_pos.shape[0]
    nlog=all_pos.shape[1]
    natom=all_pos.shape[2]
    fqt=np.zeros(nlog-1)
    qmin = 2.0*np.pi/lbox[0]
    q = int(2.*Q/qmin-1.)
    filename="./qvectors/qvector.%03d" % q
    qvec=np.loadtxt(filename)
    qvec=qvec*qmin
    qvect=np.transpose(qvec)
    fqt_list=[]
    fqt_out=[]
    error=[]
    print("nlin", nlin)
    for lin_id in range(0, nlin):
        x=all_pos[lin_id,:,:,:]
        print(lin_id)
        for i in range(0, 5):
            dr = x[1:,i,:]-x[:-1,i,:]

            for tw in range(0,1):
                if(tw==0):
                    temp=np.matmul(dr,qvect)
                    temp=np.cos(temp)
                    temp=np.mean(temp, axis=1) # averaging over all qvectors
                    fqt_list.append(list(temp)) # appending the values for each particle
                    cal=1
        sys.stdout.write("\rcompleated=%d"% (int(100.*lin_id/nlin)))
        sys.stdout.flush()
    fqt_list = np.array(fqt_list)
    return(np.mean(fqt_list, axis=0), time)

In [ ]:
fqt_log = fqt_self_log(pos_a[:10,:,:,:],l_box[0,:],Q, log_space)

In [ ]:
print(fqt_log.shape)

In [ ]:
avg = np.mean(np.array(fqt_log[0][1]))
print(avg.shape)


In [ ]:
print(fqt_log-fqt_log_2)